In [121]:
# Check cx to Django
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

from order.models import Order
ord = Order.objects.first()
ord

<Order: Nigel Samplestock: rh4 1jj by Collect From Denbies Shop>

In [256]:
from typing import List
import pandas as pd
import numpy as np
from jinja2 import Environment, FileSystemLoader
from weasyprint import HTML

In [120]:
# 1 Sheet class with manual data frame creation
# 2 Tutorial for creating PDF from dataframe https://pbpython.com/pdf-reports.html

In [260]:
class CustomerSheet():
    
    _cols: {}
    _customer_headers: []
    _product_headers: []
        
    def __init__(self):
        self._cols = {}
        self._cols['customer_headers'] = ('F-Number',)
        
    @property
    def customer_headers(self) -> List:
        self._customer_headers = []
        return self._customer_headers
    
    @property
    def headers(self) -> List:
        '''Sheet headers comprising Order details and product names'''
        headers = [self._customer_headers + self._product_headers]
        return headers
    
    
    def to_df(self):
        df = pd.DataFrame([
            # headers
            ('F-Number','Customer First Name','Customer Last Name','Customer Postcode','Customer Address','Fulfillment Method','Fulfullment Date','', 'Brocoli' ,'Celery','Veg Bag','Total'),
            # Pack size row
            ('','','','','','','','Std Pack Size', '1','1','1',''),
            # Current Price row
            ('','','','','','','','Current Price', '£1.00','£1.30','£11.00',''),
            ('20-1-001','Jon','Whittler','92, Long Acre, Dorking. Surrey.','RH4 1LD','DELIVERY','Fri 8 Jun 2020','Quantity', '1','2','1','5'),
            # Pad out rows to force next output to new page
            ('','','','','','', '','Cost','£1.00','£2.60','£11.00','£14.60'),
            ('','','','','','', '','','','','',''),
            ('','','','','','', '','','','','',''),
            ('','','','','','', '','','','','',''),
            ('','','','','','', '','','','','',''),

        ])
        df_transposed = df.transpose()
#         return df
        return df_transposed
    
    def save_xlsx(self, path='./generated_customer.xlsx'):
        df_output = self.to_df()
        writer = pd.ExcelWriter(path, engine='xlsxwriter')
        sheet_name='CUSTOMER'
        df_output.to_excel(writer, sheet_name=sheet_name)
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        worksheet.set_zoom(60)
        # Add a format. Green fill with dark green text.
        format2 = workbook.add_format({'bg_color': '#C6EFCE',
                                    'font_color': '#006100'})
        writer.save()
        
    
cs = CustomerSheet()
print(cs.save_xlsx())

None


In [271]:
# 2 Pandas to PDF tutorial using Jinja
df = cs.to_df()
from pandas.core.frame import DataFrame
assert isinstance(df,DataFrame)


# 2.1 create a template
# 2.2 Add variables to the templates context
# 2.3 render the template into HTML

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("customer_sheet.html")
template_vars = {"title" : "Farmbox Customer Sheet",
                 "customer_sheet": df.to_html()}
html_out = template.render(template_vars)
HTML(string=html_out).write_pdf("customer_sheet.pdf", stylesheets=["style.css", "customer_sheet.css"])
# print(html_out)